# 🟢 Lesson 1: The Concept of Diffusion

## 👋 Welcome!
Welcome to the **Stable Diffusion Engineering Course**. This is notebook 1 of 20.

### Goal
In this first lesson, we won't write complex code yet. We will understand the **Core Intuition** behind diffusion: **Order from Chaos**.

### What is Diffusion?
Imagine dropping a drop of ink into a glass of water. Over time, the ink *diffuses* until the water is uniformly colored. It turns from a structured state (drop) to a chaotic state (mix).

**Generative Diffusion** is the *math of reversing time*. If we could mathematically "un-mix" the water, we could get the ink drop back.

In AI:
1.  **Forward Process (Training)**: Take an image (e.g., a cat) and slowly add noise until it is pure static.
2.  **Reverse Process (Inference)**: Take pure static and ask a Neural Network to predict "what noise was added?" and subtract it.

Let's visualize this Forward Process!

In [ ]:
# 1. Setup our environment
import notebook_utils
project_root, device, dtype = notebook_utils.setup_notebook()

import torch
import numpy as np
from PIL import Image
import requests
from io import BytesIO

## 1. Load an Image
We'll download a sample image to destroy.

In [ ]:
url = "https://raw.githubusercontent.com/CompVis/stable-diffusion/main/assets/stable-samples/img2img/sketch-mountains-input.jpg"
response = requests.get(url)
image = Image.open(BytesIO(response.content)).convert("RGB").resize((512, 512))

# Display using our helper
notebook_utils.show_image(image, title="Original Image")

## 2. The Forward Diffusion (School of Destruction)

We will manually add Gaussian Noise to this image. This simulates what happens during training.

Formula: $x_t = (1 - \alpha) \cdot x_0 + \alpha \cdot \epsilon$
*(Simplified linear interpolation)*

In [ ]:
def add_noise(img, t):
    """
    Adds noise to an image.
    t: How much noise? (0.0 to 1.0)
    """
    # Convert to numpy array (0-1 range)
    img_arr = np.array(img) / 255.0
    
    # Generate random noise (same shape as image)
    noise = np.random.normal(0, 1, img_arr.shape)
    
    # Mix them
    # If t=0.0 -> Use 100% image
    # If t=1.0 -> Use 100% noise
    noisy_image = (1 - t) * img_arr + t * noise
    
    # Clip values to stay valid image (0-1)
    noisy_image = np.clip(noisy_image, 0, 1)
    
    # Convert back to PIL
    return Image.fromarray((noisy_image * 255).astype(np.uint8))

# Visualize steps
import matplotlib.pyplot as plt

steps = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]
fig, axs = plt.subplots(1, len(steps), figsize=(20, 4))

for i, t in enumerate(steps):
    n_img = add_noise(image, t)
    axs[i].imshow(n_img)
    axs[i].set_title(f"Noise {int(t*100)}%")
    axs[i].axis('off')

plt.show()

## 3. The Miracle of Reversal

Look at the last image above (100% Noise). It contains **no information** about the mountain. It is pure entropy.

However, the image at **60% noise** still has ghost-like outlines. 

**Stable Diffusion's job** is to look at the 60% version and guess what the 59% version looked like. Then 58%. Then 57%.

In the next notebook, we will load the actual libraries and see this happen!